# convert mountainlab output to pyphy

In [1]:
cd ..

/home/droumis/Src/pyphy


In [2]:
from pyphy import tools
from pyphy import mountainsort
import sys, os, json
import pandas as pd
import numpy as np
# from collections import defaultdict
sys.path.append('/home/droumis/Src/franklab_ms4/')
# import ms4_franklab_proc2py as p2p
from mountainlab_pytools import mdaio
%load_ext autoreload
%autoreload 2

In [5]:
config_path = '/home/droumis/Src/pyphy/config_virga.json'
config = tools.read_config(config_path)

animal = 'JZ1'
dates = list(config['JZ1']['day2date'].values())
dates.sort()
ntrodes = np.arange(1,31)

In [4]:
mda_preproc_dir = os.path.join(config['data_dir'], animal, 'preprocessing')
ms_firings_dir = os.path.join(config['data_dir'], animal, 'mountainlab_output')

for date in dates:
# deal with one animal_day at a time bc memory usage rate is high per worker
    out = mountainsort.get_spikes_from_mda(animal, date, ntrodes, mda_preproc_dir, ms_firings_dir, config_path)

    spikes = out['spikes']
    marks = out['marks']
    clips = out['clips']
    cluster_metrics = out['metrics']

    tools.save_df_to_pyphy(spikes, 'spikes', config_path=config_path)

    tools.save_df_to_pyphy(marks, 'marks', config_path=config_path)

    tools.save_df_to_pyphy(cluster_metrics, 'cluster_metrics', config_path=config_path)

SyntaxError: invalid syntax (<ipython-input-4-27f94a215bdc>, line 4)

don't save the entire day's clips because that file would be like another 25 GB  
`tools.save_df_to_pyphy(clips, 'clips', config_path=config_path)`

# test loading and filtering:
1.  filter cluster metrics for single unit clusters
2.  use the result to filter marks df for single unit marks based on index keys

In [45]:
datafilter = {}
datafilter['animals_days'] = {'JZ1':[1,]}
datafilter['datatypes'] = ['cluster_metrics', 'ntrodeInfo', 'marks']
# datafilter['multiunit'] = ['multiunit==0']

In [46]:
data = tools.load_data(datafilter, config_path=config_path)

loading cluster_metrics


In [47]:
marks = data['marks']
cluster_metrics = data['cluster_metrics']
ntrodeInfo = data['ntrodeInfo']

In [44]:
use_clusters = list(cluster_metrics.query('multiunit==0').index)
use_clusters = [(str(a),int(d),int(n),int(c)) for a,d,n,c in use_clusters] #cleaning

In [74]:
#drop the timedelta and epoch levels then do a cross section to get non multiunit clusters
singleunit_marks = marks[marks.index.droplevel(['epoch','timedelta']).isin(use_clusters)]

In [75]:
singleunit_marks.head()

ch00      ch01      ch02  \
animal day epoch timedelta       ntrode cluster                                 
JZ1    1   1     00:00:00.199433 2      5        0.808536  0.139140 -6.741202   
                 00:00:02.414800 2      5        1.217971  1.681590 -4.444706   
                 00:00:03.226933 2      5       -0.211903  2.876636 -4.342409   
                 00:00:03.456300 2      5        1.355214  1.476823 -3.876055   
                 00:00:03.566366 2      5        1.971174  1.117805 -5.470773   

                                                     ch03  ch04  ch05  ch06  \
animal day epoch timedelta       ntrode cluster                               
JZ1    1   1     00:00:00.199433 2      5        0.051862   NaN   NaN   NaN   
                 00:00:02.414800 2      5        0.323727   NaN   NaN   NaN   
                 00:00:03.226933 2      5        0.753980   NaN   NaN   NaN   
                 00:00:03.456300 2      5        2.675195   NaN   NaN   NaN   
                 00:00:03.566366 2      5        0.152911   NaN   NaN   NaN   

                                                 ch07  
animal day epoch timedelta       ntrode cluster        
JZ1    1   1     00:00:00.199433 2      5         NaN  
                 00:00:02.414800 2      5         NaN  
                 00:00:03.226933 2      5         NaN  
                 00:00:03.456300 2      5         NaN  
                 00:00:03.566366 2      5         NaN